In [1]:
import os
import importlib
from ipywidgets import interact, IntSlider, fixed, ToggleButton
from IPython.display import display
import edit.file_manager as fm
import edit.create as cre
import edit.filter_delete as fd
import edit.edit_func as ef
import edit.search as sea
import edit.plot as ep
import edit.interpolate as inter
importlib.reload(fm)
importlib.reload(cre)
importlib.reload(fd)
importlib.reload(ef)
importlib.reload(sea)
importlib.reload(ep)
importlib.reload(inter)
# pip install ipywidgets

# ファイルパスの設定
data_path_list = []
data_path_list.append("data/outputData/england_epl/2015-2016/2015-10-31 - 15-45 Chelsea 1 - 3 Liverpool")
data_path_list.append("data/outputData/england_epl/2015-2016/2016-02-07 - 19-00 Chelsea 1 - 1 Manchester United")
data_path_list.append("data/outputData/england_epl/2015-2016/2016-03-02 - 23-00 Liverpool 3 - 0 Manchester City")

game_num = 1
period_num = 1
data_path = os.path.join(data_path_list[game_num], f"{period_num}")
print(data_path)

# ファイルマネージャー作成
manager = fm.FileManager(data_path)

# データの読み込み
data_size_list = [[96, 93], [89, 100], [89, 89]]
data_size = data_size_list[game_num][period_num - 1]
data_list = manager.load_files("datas", 0, data_size)

# その他ファイル読み込み
reverse_numbers, set_frames, replay_frames = manager.load_any_file()
print(reverse_numbers)
# チーム反転
#fd.reverse_team(data_list, reverse_numbers)

data/outputData/england_epl/2015-2016/2016-02-07 - 19-00 Chelsea 1 - 1 Manchester United/1
Loaded 90 files
[4, 6, 13, 20, 25, 30, 34, 37, 46, 58, 60, 65, 68, 70, 72, 78, 79, 80, 81, 84, 85]


In [2]:
# フォーメーション情報の設定
formations_left, formations_right = manager.load_formation_file()
print(f"formations_left: {formations_left}")
print(f"formations_right: {formations_right}")

formations_left: {(0, 1): [['12', '17'], ['36', '27', '16', '43'], ['35', '8', '9'], ['10']]}
formations_right: {(0, 1): [['5', '26'], ['2', '12', '21', '28'], ['22', '4', '8'], ['19']]}


In [3]:
# フィルタリング
# 手動
fileter_frames = manager.load_filter_file()
print(fileter_frames)
fd.filter_data_manual(data_list, fileter_frames)

# 自動
fd.filter_data_auto(data_list)

{0: (582, 615), 5: (87, 87), 6: (1, 563), 7: (1, 466), 8: (1, 236), 9: (428, 750), 12: (134, 750), 13: (219, 750), 14: (1, 150), 17: (309, 750), 18: (490, 750), 19: (326, 644), 21: (20, 337), 23: (289, 750), 24: (1, 126), 25: (143, 493), 31: (46, 623), 32: (294, 750), 35: (1, 301), 37: (143, 249), 43: (78, 342), 44: (287, 750), 45: (1, 125), 46: (196, 750), 50: (66, 750), 51: (218, 750), 52: (1, 750), 53: (745, 745), 54: (324, 324), 56: (585, 750), 57: (1, 254), 58: (395, 750), 60: (143, 750), 61: (1, 261), 62: (79, 750), 63: (647, 750), 64: (512, 750), 65: (1, 478), 68: (72, 750), 71: (253, 614), 74: (263, 750), 76: (375, 750), 79: (500, 750), 80: (1, 209), 81: (270, 599), 87: (166, 750), 88: (1, 69)}


In [4]:
# True/False切り替えボタン配置
toggle = ToggleButton(
    value=True,
    description='True',
    icon='check'
)

def on_toggle_change(change):
    toggle.description = str(toggle.value)

toggle.observe(on_toggle_change, names='value')
print("True: 全部補正、False：一部補正")
display(toggle)

True: 全部補正、False：一部補正


ToggleButton(value=True, description='True', icon='check')

In [5]:
# 現在のフレーム
cur_data_num = 0

In [6]:
# 補正
data = data_list[cur_data_num]

if toggle.value:
    player_lists_left, player_lists_right = cre.create_player_lists(formations_left, formations_right)
    print(f"player_lists_left: {player_lists_left}")
    print(f"player_lists_right: {player_lists_right}")
    edit_func = ef.EditFunc(0, cur_data_num)
else:
    edit_func = ef.EditFunc(cur_data_num, cur_data_num)

# 背番号削除
edit_func.delete_jersey_number(data_list)

# 手動補正
edit_numbers = manager.load_edit_file()
edit_func.edit_data_manual(data_list, edit_numbers)

# 自動補正
print(f"set_frames: {set_frames}")
edit_func.edit_data_auto(data_list, player_lists_left, player_lists_right, 0.1, 20.0, 0.5, set_frames)
# print(f"player_lists_left: {player_lists_left}")
# print(f"player_lists_right: {player_lists_right}")

# 描画
try:
    last_frame_value
except NameError:
    last_frame_value = 1

def on_frame_change(change):
    global last_frame_value
    last_frame_value = change['new']

frame_slider = IntSlider(min=1, max=750, step=1, value=last_frame_value, description="Frame")
frame_slider.observe(on_frame_change, names='value')

plot_func = ep.Plot(data_path)
sea.search_none_frame(data, cur_data_num)
lims = [[-50.0, 50.0], [50.0, -50.0]]
#lims = [[-50.0, -20.0], [20.0, -20.0]]
#lims = [[20.0, 50.0], [20.0, -20.0]]
colors = ["red", "blue"]
#colors = ["blue", "red"]
interact(
    plot_func.plot_frame_data,
    data_base=fixed(data_path),
    data=fixed(data),
    data_num=fixed(cur_data_num),
    frame_num=frame_slider,
    lims=fixed(lims),
    colors=fixed(colors),
    period_num=fixed(period_num)
)

player_lists_left: {(0, 1): {'12': {'position': (-4, 2), 'location': [], 'relation': {}}, '17': {'position': (-4, 1), 'location': [], 'relation': {}}, '36': {'position': (-3, 4), 'location': [], 'relation': {}}, '27': {'position': (-3, 3), 'location': [], 'relation': {}}, '16': {'position': (-3, 2), 'location': [], 'relation': {}}, '43': {'position': (-3, 1), 'location': [], 'relation': {}}, '35': {'position': (-2, 3), 'location': [], 'relation': {}}, '8': {'position': (-2, 2), 'location': [], 'relation': {}}, '9': {'position': (-2, 1), 'location': [], 'relation': {}}, '10': {'position': (-1, 1), 'location': [], 'relation': {}}}}
player_lists_right: {(0, 1): {'5': {'position': (4, 1), 'location': [], 'relation': {}}, '26': {'position': (4, 2), 'location': [], 'relation': {}}, '2': {'position': (3, 1), 'location': [], 'relation': {}}, '12': {'position': (3, 2), 'location': [], 'relation': {}}, '21': {'position': (3, 3), 'location': [], 'relation': {}}, '28': {'position': (3, 4), 'locati

interactive(children=(IntSlider(value=1, description='Frame', max=750, min=1), Output()), _dom_classes=('widge…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>

In [30]:
# ID検索
sea.search_frame(data, cur_data_num, 97.0)
sea.search_track_id(data, cur_data_num, last_frame_value)

(97.0, 9): 619,629,662~701,703~720,722~750
track_id : [66.0, 80.0, 27.0, 104.0, 88.0, 35.0, 102.0, 82.0, 103.0, 60.0, 97.0]


In [8]:
# 画像を開く
picture_path = plot_func.display_picture(cur_data_num, last_frame_value)
!code "{picture_path}"

In [7]:
# 一度しか出現しないtrack_idを削除
fd.delete_once_track_id(data_list, edit_numbers)

In [8]:
# delete_numbersに従い、track_idを削除
delete_numbers = manager.load_delete_file()
print(delete_numbers)
fd.delete_track_id_by_frame(data_list, delete_numbers)

# 背番号被り確認
sea.search_multiple_frame(data_list, 5.0)

{7: [(601, 78.0), (667, 69.0), (685, 71.0)], 11: [(498, 34.0)], 15: [(317, 16.0)], 18: [(439, 77.0)], 20: [(288, 5.0)], 23: [(151, 5.0)], 24: [(258, 25.0), (512, 57.0)], 26: [(599, 17.0)], 27: [(489, 46.0)], 29: [(562, 51.0)], 41: [(509, 14.0)], 45: [(706, 94.0), (712, 120.0)], 47: [(262, 18.0), (391, 53.0)], 51: [(201, 51.0)], 56: [(560, 97.0)], 57: [(655, 88.0)], 58: [(332, 20.0)], 61: [(717, 72.0)], 64: [(449, 49.0)], 67: [(230, 25.0)], 69: [(427, 25.0), (705, 91.0)], 78: [(80, 44.0)], 84: [(475, 33.0)], 86: [(453, 18.0)]}
data_num: 2
287: 36, left (track_id: (39.0, {48.0}))
288: 36, left (track_id: (39.0, {48.0}))
289: 36, left (track_id: (39.0, {48.0}))
290: 36, left (track_id: (39.0, {48.0}))
291: 36, left (track_id: (39.0, {48.0}))
292: 36, left (track_id: (39.0, {48.0}))
293: 36, left (track_id: (39.0, {48.0}))
294: 36, left (track_id: (39.0, {48.0}))
295: 36, left (track_id: (39.0, {48.0}))
296: 36, left (track_id: (39.0, {48.0}))
297: 36, left (track_id: (39.0, {48.0}))
298: 

In [ ]:
# 描画
cur_data_num = 23
data = data_list[cur_data_num]
plot_func = ep.Plot(data_path)
sea.search_none_frame(data, cur_data_num)
lims = [[-50.0, 50.0], [50.0, -50.0]]
interact(
    plot_func.plot_frame_data,
    data_base=fixed(data_path),
    data=fixed(data),
    data_num=fixed(cur_data_num),
    frame_num=frame_slider,
    lims=fixed(lims),
    colors=fixed(colors),
    period_num=fixed(period_num)
)

none frame: 14,85,93,95,241,244~245,260~271,289~750


interactive(children=(IntSlider(value=110, description='Frame', max=750, min=1), Output()), _dom_classes=('wid…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>

In [13]:
# ID検索
sea.search_frame(data, cur_data_num, 106.0)
sea.search_track_id(data, cur_data_num, 484)

(106.0, 4): 543,558
track_id : [85.0, 79.0, 65.0, 73.0, 59.0, 81.0, 84.0, 44.0, 82.0, 76.0, 71.0, 66.0, 72.0, 6.0, 75.0]


In [ ]:
# フレーム検索（全体）
edit_numbers = manager.load_edit_file()
sea.search_frames(data, cur_data_num, edit_numbers)

Loaded 101 in edit file
start 1
(29.0, 43): 510~568,573,583~640
(39.0, ): 510~546,594~595,597~644,646~692,695~700,703~718
(47.0, 24): 510~560,562~564,567
(60.0, 16): 510~559,561,566,590~593,596,602~610,615
(62.0, 16): 510~511,611
(66.0, 36): 510~568,573,583~644,646~692,695~700,703~711,715~718
(67.0, 10): 510~568,573,583~644,646~692,695~700,703~718
(68.0, 8): 510~568,573,583~586
(69.0, 35): 510~542,547~568,573,583~644,646~687,689~692,695~700,703~718
(70.0, 4): 510~540,544~566,591,596,621~644,646~692,695~700,703~718
(71.0, 12): 510~536,538
(72.0, 26): 510~550,552~568,573,584,590~598,603
(73.0, 16): 510~511,515,517,519,521~524,526~528,530~532,539
(74.0, 17): 510~536,540~543,548,557~561,563~568
(75.0, 9): 510~568,573,588~592,595~616
(76.0, 19): 510~519,521~526,528~529
(77.0, 17): 510~568,573,583~644,646~682
(78.0, 16): 512,515~548,556~568,573,583~589,598,602~603,608,610~613,617,626,630,632~644,646~665
start 515
(33.0, 12): 553~560,566~568,573,583~588,591~599,602~644,646~692,695~700,703~718

In [7]:
# 複数ある背番号の統合
inter.integrate_jerseis(data_list)

# 背番号被り確認
sea.search_multiple_frame(data_list, 0.0)

In [8]:
# 時系列座標の計算
print(replay_frames)
trajectories_left, trajectories_right = cre.create_trajectories(formations_left, formations_right, replay_frames)
inter.update_trajectories(data_list, trajectories_left, "left")
inter.update_trajectories(data_list, trajectories_right, "right")
print(f"trajectories_left: {trajectories_left[(0, 1)]}")
print(f"trajectories_right: {trajectories_right[(0, 1)]}")

{0: (1, 750), 6: (564, 750), 13: (96, 218), 14: (151, 750), 18: (293, 750), 24: (127, 750), 31: (624, 750), 35: (302, 750), 41: (478, 750), 45: (594, 750), 47: (256, 750), 51: (131, 217), 53: (31, 750), 56: (483, 750), 61: (262, 750), 63: (183, 750), 65: (479, 750), 69: (79, 750), 71: (615, 750), 79: (429, 499), 80: (210, 750), 86: (147, 750), 88: (70, 750)}
trajectories_left: {'12': [(0.72, [-45.158272390253636, 28.751702844167497]), (0.76, [-49.06996634648937, 35.89808466900284]), (0.8, [-47.567907507991016, 42.319306600869844]), (0.84, [-71.63032012772038, 21.72605658834461]), (0.88, [-81.35051405364531, 20.099511613674792]), (0.9199999999999999, [-79.0615188164294, 18.05878706438393]), (0.96, [-77.11642965918719, 19.227161995534914]), (1.0, [-51.52959863502218, 37.90014453406086]), (1.04, [-47.47460066322209, 33.54474828616202]), (1.0799999999999998, [-49.14397695184763, 34.98900391990921]), (1.12, [-48.6187627079124, 34.017324208003856]), (1.1600000000000001, [-46.69585229501415, 

In [9]:
# 座標補間
inter.add_entry_to_data(data_list, trajectories_left, trajectories_right)

In [ ]:
# 描画
cur_data_num = 0
data = data_list[cur_data_num]
plot = ep.Plot(data_path)
sea.search_none_frame(data, cur_data_num)
lims = [[-50.0, 50.0], [50.0, -50.0]]
interact(
    plot.plot_frame_data,
    data_base=fixed(data_path),
    data=fixed(data),
    data_num=fixed(cur_data_num),
    frame_num=frame_slider,
    lims=fixed(lims),
    colors=fixed(colors),
    period_num=fixed(period_num)
)

none frame: None


interactive(children=(IntSlider(value=1, description='Frame', max=750, min=1), Output()), _dom_classes=('widge…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>

In [10]:
# ファイル保存
manager.save_files(data_list)

Existed files deleted
Data saved to data/outputData/england_epl/2015-2016/2015-10-31 - 15-45 Chelsea 1 - 3 Liverpool/2/edit_datas


In [ ]:
import os
from ipywidgets import interact, IntSlider, fixed, ToggleButton
from edit.file_manager import FileManager
import edit.create as cre
import edit.filter_delete as fd
from edit.edit_func import EditFunc
import edit.search as sea
from edit.plot import Plot
import edit.interpolate as inter
# pip install ipywidgets

# ファイルパスの設定
data_path_list = []
data_path_list.append("data/outputData/england_epl/2015-2016/2015-10-31 - 15-45 Chelsea 1 - 3 Liverpool")
data_path_list.append("data/outputData/england_epl/2015-2016/2016-02-07 - 19-00 Chelsea 1 - 1 Manchester United")
data_path_list.append("data/outputData/england_epl/2015-2016/2016-03-02 - 23-00 Liverpool 3 - 0 Manchester City")

game_num = 0
period_num = 1
data_path = os.path.join(data_path_list[0], f"/{period_num}")
output_path = os.path.join(data_path, f"simple_data.json")

# ファイルマネージャー作成
manager = FileManager(data_path)

# データの読み込み
data_size = 96
data_list = manager.load_files("edit_datas", 0, data_size)

In [ ]:
# 描画
cur_data_num = 0
data = data_list[cur_data_num]
plot_func = Plot(data_path)
sea.search_none_frame(data, cur_data_num)
lims = [[-50.0, 50.0], [50.0, -50.0]]
colors = ["red", "blue"]
#colors = ["blue", "red"]
interact(
    plot_func.plot_frame_data,
    data_base=fixed(data_path),
    data=fixed(data),
    data_num=fixed(cur_data_num),
    frame_num=frame_slider,
    lims=fixed(lims),
    colors=fixed(colors),
    period_num=fixed(period_num)
)

In [ ]:
# データ単純化
simple_data = cre.create_simple_data(data_list, period_num)
manager.save_file(output_path, simple_data)